In [3]:
import geopandas as gpd

# Load shapefiles
lga_gdf = gpd.read_file("LGA_POLYGON.shp")
postcode_gdf = gpd.read_file("POSTCODE_POLYGON.shp")

# Ensure CRS matches
postcode_gdf = postcode_gdf.to_crs(lga_gdf.crs)

# Input postcode
user_postcode = input("Enter a Victorian postcode: ").strip()

# Find matching postcode
matched_postcode = postcode_gdf[postcode_gdf['POSTCODE'] == user_postcode]

if matched_postcode.empty:
    print("Postcode not found in the dataset.")
else:
    # Get the centroid of the postcode area
    postcode_centroid = matched_postcode.geometry.centroid.iloc[0]

    # Find the LGA polygon that contains the centroid
    matched_lga = lga_gdf[lga_gdf.contains(postcode_centroid)]

    if matched_lga.empty:
        print("No matching LGA found for the centroid.")
    else:
        # Display LGA name
        lga_name = matched_lga.iloc[0]['LGA_NAME'] if 'LGA_NAME' in matched_lga.columns else matched_lga.iloc[0]['NAME']
        print(f"Postcode {user_postcode} falls under the council: {lga_name}")


Enter a Victorian postcode: 3141
Postcode 3141 falls under the council: STONNINGTON


In [4]:
import geopandas as gpd
import pandas as pd

# Load shapefiles
lga_gdf = gpd.read_file("LGA_POLYGON.shp")
postcode_gdf = gpd.read_file("POSTCODE_POLYGON.shp")

# Ensure CRS matches
postcode_gdf = postcode_gdf.to_crs(lga_gdf.crs)

# Calculate centroids for postcode polygons
postcode_gdf['centroid'] = postcode_gdf.geometry.centroid

# Prepare result list
results = []

# Loop through postcode areas
for idx, row in postcode_gdf.iterrows():
    postcode = row['POSTCODE']
    centroid = row['centroid']

    # Find LGA containing the centroid
    matched_lgas = lga_gdf[lga_gdf.contains(centroid)]

    for _, lga_row in matched_lgas.iterrows():
        lga_name = lga_row.get('LGA_NAME') or lga_row.get('NAME') or "Unknown"
        lga_code = lga_row.get('LGA_CODE', "")
        results.append({
            'postcode': postcode,
            'centroid_lat': centroid.y,
            'centroid_lon': centroid.x,
            'lga_name': lga_name,
            'lga_code': lga_code
        })

# Convert to DataFrame
result_df = pd.DataFrame(results)

# Save to CSV
result_df.to_csv("vic_postcode_lga_mapping.csv", index=False)

print("✅ CSV saved as vic_postcode_lga_mapping.csv")


✅ CSV saved as vic_postcode_lga_mapping.csv


In [8]:
import pandas as pd
import re

# Load data
postcode_df = pd.read_csv("vic_postcode_lga_mapping.csv")
council_df = pd.read_csv("vic_council_details_rows.csv")

# Normalize function for council names
def normalize_council_name(name):
    name = name.upper().strip()
    name = re.sub(r'\s+(CITY|SHIRE|RURAL CITY)\s+COUNCIL', '', name)
    name = name.replace('BOROUGH OF ', '')
    name = name.replace('CITY COUNCIL', '').replace('SHIRE COUNCIL', '')
    name = name.replace('RURAL CITY COUNCIL', '')
    name = name.strip()
    return name

# Create matching keys
council_df['council_key'] = council_df['council_name'].apply(normalize_council_name)
postcode_df['lga_key'] = postcode_df['lga_name'].str.strip().str.upper()

# Get list of unique keys
postcode_keys = set(postcode_df['lga_key'].unique())
council_keys = set(council_df['council_key'].unique())

# Find unmatched LGA names
unmatched_lgas = postcode_keys - council_keys

# Print unmatched
if unmatched_lgas:
    print("⚠️ The following LGA names from the postcode data had no match in the council file:\n")
    for lga in sorted(unmatched_lgas):
        print(f" - {lga}")
else:
    print("✅ All LGA names matched successfully!")

# Merge the datasets
merged_df = pd.merge(postcode_df, council_df, left_on='lga_key', right_on='council_key', how='left')
df_selected = merged_df.rename(columns={"postcode_x": "postcode"})[
    ["council_name", "tel_phone", "ceo_name", "postcode",
     "fax", "mayor", "email", "website", "address"]
]
# Save to CSV
df_selected.to_csv("postcode_to_council_merged.csv", index=False)
print("✅ CSV saved as postcode_to_council_merged.csv")


⚠️ The following LGA names from the postcode data had no match in the council file:

 - FRENCH-ELIZABETH-SANDSTONE ISLANDS (UNINC)
 - YARRIAMBIACK
✅ CSV saved as postcode_to_council_merged.csv


In [10]:
df_selected.shape

(693, 9)